In [1]:
import sklearn.datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from collections import Counter
import pandas as pd

In [2]:
redwine_data = pd.read_csv('../data/winequality-red.csv',sep=';')
# redwine_data = pd.read_csv('../data/winequality-white.csv',sep=';')
Redwine_datas = redwine_data.values
x = Redwine_datas[:,:11]
y = Redwine_datas[:,11]
Counter(y)

Counter({5.0: 681, 6.0: 638, 7.0: 199, 4.0: 53, 8.0: 18, 3.0: 10})

In [3]:
x = torch.from_numpy(x).type(torch.FloatTensor)
y = torch.from_numpy(y).type(torch.LongTensor)

In [4]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

In [5]:
class MyClassifier(nn.Module):
    def __init__(self, first, second, third):
        super(MyClassifier,self).__init__()
        self.fc1 = nn.Linear(11,first)
        self.fc2 = nn.Linear(first,second)
        self.fc3 = nn.Linear(second, third)
        self.fc4 = nn.Linear(third,2)
        
    def forward(self,x):
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.fc2(x)
        x = torch.tanh(x)
        x = self.fc3(x)
        x = torch.tanh(x)
        x = self.fc4(x)
        return x
             
    def predict(self,x):
        pred = F.softmax(self.forward(x), dim=0)
        ans = []
        for t in pred:
            if t[0]>t[1]:
                ans.append(0)
            else:
                ans.append(1)
        return torch.tensor(ans)

In [6]:
# ideal_params = [(11, 11),(22, 11),(22, 22),(6, 6),(11, 6),(22, 6)]
ideal_params = [(11, 11,11),(6, 6,6),(11, 6, 6)]

In [7]:
for ideal_param in ideal_params:
    model = MyClassifier(*ideal_param)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    epochs = 20000
    losses = []
    train_accuracy = []
    test_accuracy = []
    
    for i in range(epochs):
        x_pred = model.forward(x_train)
        loss = criterion(x_pred,y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i%50 == 0:
            losses.append(loss.item())
            train_accuracy.append(accuracy_score(model.predict(x_train), y_train))
            test_accuracy.append(accuracy_score(model.predict(x_test), y_test))
    
    min_loss = min(losses)
    max_train = max(train_accuracy)
    max_test = max(test_accuracy)
    
    print('with params {}: minloss:{}  maxtrain:{}  maxtest:{}'.format(ideal_param, min_loss, max_train, max_test))



with params (11, 11, 11): minloss:0.2093474119901657  maxtrain:0.8230563002680965  maxtest:0.7541666666666667
with params (6, 6, 6): minloss:0.42907634377479553  maxtrain:0.7864164432529044  maxtest:0.75
with params (11, 6, 6): minloss:0.35097581148147583  maxtrain:0.803395889186774  maxtest:0.75


In [ ]:
# red 1w epoch
# with params (11, 11): minloss:0.23819774389266968  maxtrain:0.8909740840035746  maxtest:0.7395833333333334
# with params (22, 11): minloss:0.17275948822498322  maxtrain:0.9186773905272565  maxtest:0.74375
# with params (22, 22): minloss:0.08662816882133484  maxtrain:0.9088471849865952  maxtest:0.73125
# with params (6, 6): minloss:0.3797895610332489  maxtrain:0.7890974084003575  maxtest:0.73125
# with params (11, 6): minloss:0.33057165145874023  maxtrain:0.80875781948168  maxtest:0.7520833333333333
# with params (22, 6): minloss:0.26135367155075073  maxtrain:0.8168007149240393  maxtest:0.7208333333333333

# white 1w epoch
# with params (11, 11): minloss:0.4417298436164856  maxtrain:0.7698366394399067  maxtest:0.7360544217687075
# with params (22, 11): minloss:0.4224976599216461  maxtrain:0.764585764294049  maxtest:0.7482993197278912
# with params (22, 22): minloss:0.41760051250457764  maxtrain:0.8004667444574096  maxtest:0.7761904761904762
# with params (6, 6): minloss:0.4764550030231476  maxtrain:0.7517502917152858  maxtest:0.7414965986394558

# red 1w epoch
# with params (11, 11, 11): minloss:0.3271178603172302  maxtrain:0.8203753351206434  maxtest:0.7416666666666667
# with params (6, 6, 6): minloss:0.34230032563209534  maxtrain:0.7935656836461126  maxtest:0.7395833333333334
# with params (11, 6, 6): minloss:0.32185521721839905  maxtrain:0.8445040214477212  maxtest:0.7666666666666667

# red 2w epoch
# with params (11, 11, 11): minloss:0.2093474119901657  maxtrain:0.8230563002680965  maxtest:0.7541666666666667
# with params (6, 6, 6): minloss:0.42907634377479553  maxtrain:0.7864164432529044  maxtest:0.75
# with params (11, 6, 6): minloss:0.35097581148147583  maxtrain:0.803395889186774  maxtest:0.75